In [61]:
import pandas as pd
import pyodbc
import numpy as np

In [62]:
import warnings
warnings.filterwarnings("ignore")

In [63]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                        "Server=192.168.5.18\CROPNUT;"
                        "Database=cropnuts;"
                        "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [64]:
settings = pd.read_csv("Settings.csv")

In [65]:
settings = settings.loc[settings['client_type'] == "FarmLab"]

In [66]:
settings

,Unnamed: 0,use_nutrient_req,use_YieldvsSoilTestRanges,crop_code,chemical_code,client_id,farm_code,client_type,crop_name,chemical_name,client_name,farm_name
1503098,0,False,True,194,19,7669,0,FarmLab,Millet,P,OCP_Nigeria1- NW,NaN
1503099,1,False,True,194,19,7670,0,FarmLab,Millet,P,OCP_Ghana_1,NaN
1503100,2,False,True,23,19,7669,0,FarmLab,ma,P,OCP_Nigeria1- NW,NaN
1503101,3,False,True,46,19,8763,0,FarmLab,Maize,P,OCP_Nigeria3-SS/SE,NaN
1503102,4,False,True,46,19,7669,0,FarmLab,Maize,P,OCP_Nigeria1- NW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1504900,417,False,True,1651,1,38912,0,FarmLab,Sorghum (Hybrid),K,Nigeria,NaN
1504901,418,False,True,1650,19,38912,0,FarmLab,Sorghum (OPV),P,Nigeria,NaN
1504902,419,False,True,1650,1,38912,0,FarmLab,Sorghum (OPV),K,Nigeria,NaN
1504903,420,False,True,1646,19,38912,0,FarmLab,Tomatoes (Hybrid),P,Nigeria,NaN


In [67]:
len(np.array(settings)[0])

12

In [68]:
settings = settings[["client_id","client_name","farm_name","client_type","crop_code","crop_name","chemical_name","chemical_code","use_nutrient_req","use_YieldvsSoilTestRanges"]]

In [69]:
settings['UsesYieldVsSoilTestRange'] = settings['use_YieldvsSoilTestRanges']

In [70]:
settings = settings.rename(columns={"use_YieldvsSoilTestRanges":"yieldTargetsMatchesMasterCrop"})

In [71]:
arr = []

# Analysis for phosphorous

In [72]:
for record in np.array(settings):
    usesYieldVsSoilTestRange = record[10]
    if(usesYieldVsSoilTestRange == False):
        continue
    client = record[0]
    crop = record[4]
    print(client)
    print(crop)
    df = pd.read_sql(f"SET NOCOUNT ON;execute [load_YieldvsSoilTestRanges] @crop_code={crop},@Chemical_code=19,@client_id={client}", con=conn_lims)
    master = pd.read_sql(f"SET NOCOUNT ON;execute [load_YieldvsSoilTestRanges] @crop_code={crop},@Chemical_code=19,@client_id=0", con=conn_lims)
    
    df = df[df.columns.to_list()[11:]]
    master = master[master.columns.to_list()[11:]]
    
    if(df.equals(master) == True):
        record[9] = True
    else:
        record[9] = False
    
    arr.append(record)


7669
194
7670
194
7669
23
8763
46
7669
46
8761
46
8761
302
8761
24
8761
58
8761
194
8763
194
8762
194
8734
302
8762
302
8763
302
7669
302
7670
302
8730
302
8734
24
8762
24
8763
24
8730
24
8734
62
8734
58
8734
194
8734
46
7670
58
8730
46
8730
62
8730
58
8730
194
9603
58
9603
62
9603
46
10120
46
10120
720
10120
302
10120
66
10120
397
10120
46
10120
46
7669
46
10120
302
10120
720
10120
66
10120
397
10535
46
10535
62
10534
62
10534
302
10534
58
10120
62
11296
46
11296
702
11298
46
11298
58
11298
702
11298
65
11296
65
11299
702
11383
46
7669
702
7669
720
8762
975
7670
702
8762
140
7669
975
7670
720
8762
46
10120
46
10120
46
10120
46
10120
46
10120
46
8763
46
8763
46
10120
62
12449
722
10120
1410
10120
729
12449
702
8762
62
7669
62
7669
58
7669
24
8761
62
11296
397
11298
397
10120
46
7670
46
7670
62
8734
23
8761
23
8762
23
8763
23
7670
24
8757
24
8762
46
8762
58
8763
58
8762
62
8763
62
7669
154
8763
154
9603
194
8734
702
12449
46
11299
46
7669
1382
8762
33
8762
72
8762
1382
8762
720
8762
763

24106
1519
24106
1454
24106
49
24106
49
24106
64
24106
1528
24106
1528
24106
1529
24106
1529
24106
1528
24106
202
24106
153
24106
1537
24106
1224
24106
35
24106
35
24106
671
24106
1639
24106
1639
24106
1639
24106
1639
24106
1643
24106
1643
24106
1643
24106
1643
24106
33
24106
720
24106
1630
24106
1640
24106
202
24106
364
24106
1480
24106
504
24106
1538
24106
937
24106
18
24106
702
24106
202
24106
1235
24106
1529
24106
46
24106
1382
24106
676
24106
676
24106
651
24106
1640
24106
729
24106
729
24106
975
24106
140
24106
223
24106
32
24106
33
24106
1383
24106
763
24106
720
24106
327
24106
1224
24106
1112
24106
194
24106
611
24106
153
24106
58
24106
62
24106
66
24106
154
24106
72
24106
722
24106
393
24106
393
24106
328
24106
975
24106
1108
24106
1535
24106
1535
24106
81
24106
81
24106
1642
24106
938
24106
938
24106
1641
24106
1641
24106
1641
24106
1647
24106
1647
24106
1647
24106
1647
24106
46
24106
722
24106
46
24106
722
24106
702
24106
79
24106
110
24106
397
24106
63
24106
32
24106
63
241

38912
1519
38912
1519
38912
1454
38912
49
38912
49
38912
64
38912
1528
38912
1528
38912
1529
38912
1529
38912
1528
38912
202
38912
153
38912
1537
38912
1224
38912
35
38912
35
38912
671
38912
1639
38912
1639
38912
1639
38912
1639
38912
1643
38912
1643
38912
1643
38912
1643
38912
33
38912
720
38912
1630
38912
1640
38912
202
38912
364
38912
1480
38912
504
38912
1538
38912
937
38912
18
38912
702
38912
202
38912
1235
38912
1529
38912
46
38912
1382
38912
676
38912
676
38912
651
38912
1640
38912
729
38912
729
38912
975
38912
140
38912
223
38912
32
38912
33
38912
1383
38912
763
38912
720
38912
327
38912
1224
38912
1112
38912
194
38912
611
38912
153
38912
58
38912
62
38912
66
38912
154
38912
72
38912
722
38912
393
38912
393
38912
328
38912
975
38912
1108
38912
1535
38912
1535
38912
81
38912
81
38912
1642
38912
938
38912
938
38912
1641
38912
1641
38912
1641
38912
1647
38912
1647
38912
1647
38912
1647
38912
46
38912
722
38912
46
38912
722
38912
702
38912
79
38912
110
38912
397
38912
63
38912
32
3

In [73]:
arr = np.array(arr)

In [74]:
arr

array([[7669, 'OCP_Nigeria1- NW', nan, ..., False, False, True],
       [7670, 'OCP_Ghana_1', nan, ..., False, False, True],
       [7669, 'OCP_Nigeria1- NW', nan, ..., False, False, True],
       ...,
       [38912, 'Nigeria', nan, ..., False, False, True],
       [38912, 'Nigeria', nan, ..., False, False, True],
       [38912, 'Nigeria', nan, ..., False, False, True]], dtype=object)

In [75]:
df = pd.DataFrame(arr, columns = settings.columns)

In [76]:
df.client_type.value_counts()

FarmLab    1791
Name: client_type, dtype: int64

In [77]:
df.to_csv("FarmLab.csv")